# 特徴量作成

In [2]:
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import os 
import glob
import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


# 本人にアノテーションされた性格特性スコアの算出
def calc_persona(filename):
    df = pd.read_excel('../../data/Hazumi1911/questionnaire/1911questionnaires.xlsx', sheet_name=4, index_col=0, header=1)
    data = df.loc[filename, :].values.tolist()
    return [data[0]+(8-data[5]), (8-data[1])+data[6], data[2]+(8-data[7]), data[3]+(8-data[8]), data[4]+(8-data[9])]

# 第三者にアノテーションされた性格特性スコアの算出
def calc_thirdpersona(filename):
    df = pd.read_excel('../../data/Hazumi1911/questionnaire/220818thirdbigfive-Hazumi1911.xlsx', sheet_name=5, header=1, index_col=0)
    data = df.loc[filename].values.tolist()
    return [data[5], data[13], data[21], data[29], data[37]]


### ダンプファイル

In [3]:
audio = {}
text = {}
visual = {} 

third_persona = {}
persona = {}
TS_ternary = {}
SS_ternary = {}
third_sentiment = {}
sentiment = {}

vid = []

path = '../../data/Hazumi1911/dumpfiles/*'

files = glob.glob(path)

for file_path in sorted(files):
    filename = os.path.basename(file_path).split('.', 1)[0]
    df = pd.read_csv(file_path)

    vid.append(filename)
    text[filename] = df.loc[:, 'word#0001':'su'].values.tolist()
    audio[filename] = df.loc[:, 'pcm_RMSenergy_sma_max':'F0_sma_de_kurtosis'].values.tolist()
    visual[filename] = df.loc[:, '17_acceleration_max':'AU45_c_mean'].values.tolist()

    persona[filename] = calc_persona(filename)
    third_persona[filename] = calc_thirdpersona(filename)
    TS_ternary[filename] = df.loc[:, 'TS_ternary'].values.tolist()
    SS_ternary[filename] = df.loc[:, 'SS_ternary'].values.tolist()
    third_sentiment[filename] = df.loc[:, 'TS1':'TS5'].mean(axis='columns').values.tolist()
    sentiment[filename] = df.loc[:, 'SS'].values.tolist()

# ファイル書き込み
with open('../../data/Hazumi_features/Hazumi1911_features.pkl', mode='wb') as f:
    pickle.dump((SS_ternary, TS_ternary, sentiment, third_sentiment, persona, third_persona, text, audio, visual, vid), f)

In [9]:
sum = [0, 0, 0]
for value in TS_ternary.values():
    for v in value:
        sum[int(v)] += 1

sum 

[178, 881, 1380]